# Figure 3 - Analysis

In [ ]:
import sys
sys.path.append("..")

from main import *

import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr

plt.rcParams['font.size'] = 6

if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu" 
print(device)

In [ ]:
vertices = np.load('../Files/vertices_ellipse.npy').astype('float')
order = np.argsort(vertices[:, 2])
vertices = vertices[order]
eigenmodes = np.load('../Files/eigenmodes_ellipse.npy')[order]

# Individual eigenmode-gradient correlations vs connectivity radius $h$

Simulations were conducted externally on a computer cluster (`Compute Canada`) and the results are loaded here for analysis. See the script in the `ComputeCanada/kernel_size/` folder for a code example.

In [ ]:
correlations = np.load('../ComputeCanada/kernel_size/mode_correlations_per_h_2500_zoom.npy')

In [ ]:
avg_correlations = np.mean(np.abs(correlations), axis=1)

for i, a in enumerate(avg_correlations):
    plt.imshow(a, cmap='Reds', vmin=0, vmax=1)
    plt.xticks([])
    plt.yticks([])
    #plt.savefig('Matrices/correlations_{}.png'.format(i+1))
    plt.show()

In [ ]:
diagonals = []
for a in avg_correlations:
    diagonals.append(np.diag(a))

In [ ]:
diagonals_kernelsize = np.stack(diagonals, axis=1)

In [ ]:
plt.figure(figsize=(2, 2), dpi=300)
plt.imshow(diagonals_kernelsize, vmin=0, vmax=1, cmap='Reds', aspect='auto')
cbar = plt.colorbar()
cbar.ax.set_ylabel('Mode correspondence $|r|$', rotation=270, ha='center', va='center')

plt.xlabel('Neighborhood size $h$')
plt.ylabel('Mode #')
plt.yticks([0, 49], [1, 50])

# Computing variograms to estimate eigenmode wavelengths

In [ ]:
from scipy.signal import find_peaks
from scipy.stats import zscore

In [ ]:
def compute_pairwise_distances(points):
    diff = points[:, np.newaxis, :] - points[np.newaxis, :, :]
    distances = np.sqrt(np.sum(diff**2, axis=-1))
    return distances

def compute_variogram(coordinates, values, bins=np.linspace(0, 1, 30), subsample=1000, iters=10):
    variograms = []
    for _ in range(iters):
        random_ids = np.arange(coordinates.shape[0])
        np.random.shuffle(random_ids)
        random_ids = np.sort(random_ids[:subsample])
        d = compute_pairwise_distances(coordinates[random_ids])
        sub_values = values[random_ids]
        variances = (sub_values[np.newaxis, :] - sub_values[:, np.newaxis]) ** 2
        variogram = np.zeros((len(bins) - 1, ))
        for i in range(len(variogram)):
            variogram[i] = np.mean(variances[(d >= bins[i]) & (d < bins[i + 1])])
        variograms.append(variogram)
    return np.mean(np.stack(variograms, axis=0), axis=0)

def get_wavelength(variogram, bins):
    result = find_peaks(variogram)
    if any(result[0]):
        wavelength = 2 * (bins[find_peaks(variogram)[0][0]] + ((bins[1] - bins[0]) / 2))
        return wavelength
    else:
        return 2 * bins[-1]

In [ ]:
bins = np.linspace(0, 1, 60, endpoint=True)

In [ ]:
compute = False
if compute:
    wavelengths, variograms = [], []
    for i in tqdm(range(eigenmodes.shape[1])):
        variogram = compute_variogram(vertices, zscore(eigenmodes[:, i]), bins=bins, subsample=2500)
        variograms.append(variogram)
        wavelengths.append(get_wavelength(variogram, bins))
    np.save('../Results/mode_variograms.npy', variograms)
    np.save('../Results/mode_wavelengths.npy', wavelengths)
else:
    variograms = np.load('../Results/mode_variograms.npy')
    wavelengths = np.load('../Results/mode_wavelengths.npy')

# Identifying eigenmode cutoff points

In [ ]:
from scipy.ndimage import gaussian_filter1d

def filter_matrix_rows(matrix, sigma=1):
    if sigma != 0:
        filtered = np.copy(matrix)
        for i in range(filtered.shape[0]):
            filtered[i] = gaussian_filter1d(filtered[i], sigma=sigma)
        return filtered
    else:
        return matrix

def identify_mode_cutoff_points(diagonals_matrix, sigma=0):

    matrix = filter_matrix_rows(diagonals_matrix, sigma=sigma)
    
    h_cutoff_array, r_cutoff_array = [], []    
    for element in matrix:
        diff_array = []
        for i in range(1, len(matrix[0] + 1)):
            diff = element[i] - element[i - 1]
            diff_array.append(diff)
        max_diff = min(diff_array)
        cutoff_index = diff_array.index(max_diff)
        while diff_array[cutoff_index - 1] > 0 and diff_array[cutoff_index - 1] > 0:
            diff_array[cutoff_index] = 0
            max_diff = min(diff_array)
            cutoff_index = diff_array.index(max_diff)
        h_cutoff_array.append(cutoff_index)
        r_cutoff_array.append((element[cutoff_index] + element[cutoff_index + 1]) / 2)

    return np.array(h_cutoff_array), np.array(r_cutoff_array)

In [ ]:
h_values = np.linspace(0.025, 0.6, 100)

In [ ]:
h_cutoffs, r_cutoffs = identify_mode_cutoff_points(diagonals_kernelsize, sigma=1)
h_cutoffs = h_values[h_cutoffs]

In [ ]:
x = h_cutoffs[1:]
y = np.array(wavelengths[2:51])

In [ ]:
fig, ax = plt.subplots(figsize=(2, 2), dpi=300)
ax.scatter(x, y, s=2, color='black')
ax.spines[['top', 'right']].set_visible(False)
a, b = np.polyfit(x, y, deg=1)
ax.plot(x, a * x + b, linewidth=1, color='red')
ax.set_xlabel('$h$ cutoff')
ax.set_ylabel('Mode wavelength')

print(a, b)

In [ ]:
pearsonr(x, y)[0] ** 2

In [ ]:
h_cutoffs_kernel = np.copy(h_cutoffs)
r_cutoffs_kernel = np.copy(r_cutoffs)

In [ ]:
np.save('../Results/figure3_cutoffs_kernelsize.npy', h_cutoffs_kernel)

# Individual eigenmode-gradient correlations vs edge swaps

Simulations were conducted externally on a computer cluster (`Compute Canada`) and the results are loaded here for analysis. See the script in the `ComputeCanada/edge_swapping/` folder for a code example.

In [ ]:
D_mean = np.load('../ComputeCanada/edge_swapping/avg_d_per_rho_swaps.npy')
mode_similarities = np.load('../ComputeCanada/edge_swapping/mode_correlations_per_rho_swaps.npy')

fractions = np.linspace(0, 0.99, 36, endpoint=True)

scores = np.zeros((mode_similarities.shape[0], mode_similarities.shape[1]))
for i in range(mode_similarities.shape[0]):
    for j in range(mode_similarities.shape[1]):
        scores[i, j] = np.mean(np.abs(np.diag(mode_similarities[i, j])))

In [ ]:
avg_correlations = np.mean(np.abs(mode_similarities), axis=1)

for i, a in enumerate(avg_correlations):
    plt.imshow(a, cmap='Blues', vmin=0, vmax=1)
    plt.xticks([])
    plt.yticks([])
    #plt.savefig('correlations_{}.png'.format(i+1))
    plt.show()

In [ ]:
diagonals = []
for a in avg_correlations:
    diagonals.append(np.diag(a))
diagonals_edgeswaps = np.stack(diagonals, axis=1)

In [ ]:
plt.figure(figsize=(2, 2), dpi=300)
plt.imshow(diagonals_edgeswaps, vmin=0, vmax=1, cmap='Blues', aspect='auto')

cbar = plt.colorbar()
cbar.ax.set_ylabel('Mode correspondence $|r|$', rotation=270, ha='center', va='center')
plt.xlabel('Number of edge swaps ($10^3$)')
plt.ylabel('Mode #')
plt.xticks([0, 30])
plt.yticks([0, 49], [1, 50])


In [ ]:
rho_cutoffs, r_cutoffs = identify_mode_cutoff_points(diagonals_edgeswaps, sigma=1)
rho_cutoffs = fractions[rho_cutoffs]

cutoffs = []
for i, d in enumerate(diagonals_edgeswaps):
    cutoffs.append(N_swaps[np.where(d > 0.2)[0][-1]])

In [ ]:
x = np.array(rho_cutoffs[1:])
y = np.array(wavelengths[2:51])

In [ ]:
fig, ax = plt.subplots(figsize=(2, 2), dpi=300)
ax.scatter(x, y, s=2, color='black')
ax.spines[['top', 'right']].set_visible(False)
a, b = np.polyfit(x, y, deg=1)
ax.plot(x, a * x + b, linewidth=1, color='red')
ax.set_xlabel('# swaps cutoff')
ax.set_ylabel('Mode wavelength')

print(a, b)

In [ ]:
pearsonr(x, y)[0] ** 2

In [ ]:
rho_cutoffs_swaps = np.copy(rho_cutoffs)
r_cutoffs_swaps = np.copy(r_cutoffs)

In [ ]:
np.save('../Results/figure3_cutoffs_swaps.npy', rho_cutoffs_swaps)

# Plotting line profiles of previous arrays (Supplementary analysis)

Normalizing and aligning the rows of the `diagonals_kernelsize` and `diagonals_edgeswaps` arrays, related to Supplementary Figure S6.

In [ ]:
normalized_profiles_kernelsize = []
for d in diagonals_kernelsize:
    normalized_profiles_kernelsize.append(normalize(gaussian_filter1d(d, 2)))

midpoints_kernelsize = []
for p in normalized_profiles_kernelsize:
    midpoints_kernelsize.append(np.where(p[10:] < 0.5)[0][0] + 10)

aligned_curves_kernelsize = []
for i in range(len(normalized_profiles_kernelsize)):
    m = midpoints_kernelsize[i]
    segment_left = normalized_profiles_kernelsize[i][:m]
    segment_right = normalized_profiles_kernelsize[i][m:]
    aligned_curve = np.zeros((len(normalized_profiles_kernelsize[0]), ))
    if len(segment_left) > 50:
        aligned_curve[:50] = segment_left[-50:]
    else:
        aligned_curve[50 - len(segment_left):50] = segment_left
    if len(segment_right) > 50:
        aligned_curve[50:] = segment_right[:50]
    else:
        aligned_curve[50:50+len(segment_right)] = segment_right
    max_id = np.argmax(aligned_curve)
    aligned_curve[:max_id] = np.nan
    aligned_curves_kernelsize.append(aligned_curve)

In [ ]:
for d in normalized_profiles_kernelsize:
    plt.plot(d)
plt.show()

In [ ]:
for d in aligned_curves_kernelsize:
    plt.plot(d)
plt.show()

In [ ]:
normalized_profiles_edgeswaps = []
for d in diagonals_edgeswaps:
    normalized_profiles_edgeswaps.append(normalize(gaussian_filter1d(d, 2)))

midpoints_edgeswaps = []
for p in normalized_profiles_edgeswaps:
    midpoints_edgeswaps.append(np.where(p[7:] < 0.5)[0][0] + 7)

aligned_curves_edgeswaps = []
for i in range(len(normalized_profiles_edgeswaps)):
    m = midpoints_edgeswaps[i]
    segment_left = normalized_profiles_edgeswaps[i][:m]
    segment_right = normalized_profiles_edgeswaps[i][m:]
    aligned_curve = np.zeros((len(normalized_profiles_edgeswaps[0]), ))
    if len(segment_left) > 18:
        aligned_curve[:18] = segment_left[-18:]
    else:
        aligned_curve[18 - len(segment_left):18] = segment_left
    if len(segment_right) > 18:
        aligned_curve[18:] = segment_right[:18]
    else:
        aligned_curve[18:18+len(segment_right)] = segment_right
    max_id = np.argmax(aligned_curve)
    aligned_curve[:max_id] = np.nan
    aligned_curves_edgeswaps.append(aligned_curve)

In [ ]:
for d in diagonals_edgeswaps:
    plt.plot(normalize(gaussian_filter1d(d, 2)))
plt.show()

In [ ]:
for d in aligned_curves_edgeswaps:
    plt.plot(d)
plt.show()

In [ ]:
for d in aligned_curves_edgeswaps:
    plt.plot(np.linspace(0, 1, len(d)), d, color='black')

for d in aligned_curves_kernelsize:
    plt.plot(np.linspace(0, 1, len(d)), d, color='red')
plt.show()

In [ ]:
np.save('../Results/figure3_profiles_kernelsize.npy', normalized_profiles_kernelsize)
np.save('../Results/figure3_aligned_profiles_kernelsize.npy', aligned_curves_kernelsize)
np.save('../Results/figure3_profiles_edgeswaps.npy', normalized_profiles_edgeswaps)
np.save('../Results/figure3_aligned_profiles_edgeswaps.npy', aligned_curves_edgeswaps)